In [141]:
from bs4 import BeautifulSoup
import time
import sys
import codecs

# Get one html file from dir all-authors-html/ based on its appendix number, i.e., author_199.html
# Extract all the search results:
#     FirstName, LastName, 
#     NickName_1_FN, NickName_1_LN, 
#     NickName_2_FN, NickName_2_LN, 
#     NickName_3_FN, NickName_3_LN, 
#
#     Subject
#     Number_Of_Documents
# 
#     Scopus_ID
#     Affiliation(University)
#     City
#     Country
#
# Store result line-by-line into an csv file into dir authors_csv/ with naming convention: 
#     FirstName_LastName_HtmlFileNumber.csv

authorId = []
authorNm = []
authorNickNames = []
documents = []
subjects = []
affiliations = []
city = []
country = []

authorUn = []



def htmlToCsv(htmlnumber):
    html_string = "all-authors-html/author_" + str(htmlnumber) + ".html"
    try:                         
        html_doc = open(html_string,'r').read()
        soup = BeautifulSoup(html_doc, 'html.parser')
        
        st1 = soup.find("input", {"name":"st1"}) 
        if st1 is not None:
            st2 = soup.find("input", {"name":"st2"}) # found firstname and lastname  
            lastname = st1.get('value')
            firstname = st2.get('value')
            
            filename = "all-authors-csv-full-detail-data/"+firstname + "_" + lastname + "_" + str(htmlnumber) + ".csv"
            target_file = open(filename, 'w')
            
            for a in soup.find_all('a'):
                if a.get('title') == 'View this author\'s profile':
                    href = a.get('href')
                    index_start = href.index('Id')
                    index_end = href.index('&origin')
                    authorId.append(href[index_start+3:index_end])
                    authorNm.append(a.getText()[1:].replace(',',' '))
            
            for uni in soup.findAll("div", { "class" : "dataCol2" }): # nicknames
                try:
                    authorNickNames.append(uni.getText())
                except UnicodeEncodeError:
                    authorNickNames.append(uni.getText().encode('ascii', 'ignore'))
             
            for uni in soup.findAll("div", { "class" : "dataCol3" }): # number of documents
                documents.append(uni.getText())
            
            for uni in soup.findAll("div", { "class" : "dataCol4" }): # subject
                subjects.append(uni.getText())
                
            for uni in soup.findAll("div", { "class" : "dataCol5" }): # affiliation
                try:
                    affiliations.append(str(uni.getText())[1:-1])
                except UnicodeEncodeError:
                    affiliations.append(uni.getText().encode('ascii', 'ignore')[1:-1])
                    
            for uni in soup.findAll("div", { "class" : "dataCol6" }): # city
                try:
                    city.append(str(uni.getText())[1:-1])
                except UnicodeEncodeError:
                    city.append(uni.getText().encode('ascii','ignore')[1:-1])
                
            for uni in soup.findAll("div", { "class" : "dataCol7" }): # county
                country.append(str(uni.getText())[1:-1])  
                
            authornames = []
            for n in authorNickNames:
                nicknames = []
                try:
                    names = str(n.strip()).splitlines()
                except UnicodeEncodeError:
                    names = n.strip().encode('ascii','ignore').splitlines()
                for i in names:
                    if i != '':
                        nicknames.append(i.replace(',',' '))            
                if len(nicknames) == 0:
                    nicknames.append('')
                    nicknames.append('')
                    nicknames.append('')
                if len(nicknames) == 1:
                    nicknames.append('')
                    nicknames.append('')
                if len(nicknames) == 2:
                    nicknames.append('')        
                authornames.append(nicknames)
            
            numberOfDoc = []
            for n in documents:
                doc = str(n).splitlines()
                for i in doc:
                    if i!='' and i!='Documents':
                        numberOfDoc.append(i)

            subject_arr = []
            for n in subjects:
                sub = str(n).splitlines()
                one_auth_sub = ''
                
                for idx, val in enumerate(sub):                                        
                    if val != ';' and val != '':
                        if idx == len(sub)-1:
                            one_auth_sub += val.replace('; ...','')
                        else:
                            one_auth_sub += val + ' '
                subject_arr.append(one_auth_sub)
            
            for i in xrange(0, len(authorId)):
                authId = authorId[i]
                try:
                    authorName = str(authorNm[i])
                except UnicodeEncodeError:
                    authorName = authorNm[i].encode('ascii','ignore')
                nicknames_arr = authornames[i]
                numOfDoc = numberOfDoc[i]
                subject_str = subject_arr[i].replace(',',' ')
                aff = affiliations[i]
                city_name = city[i]
                count_name = country[i]
                line = authId  + ',' + authorName + ',' + nicknames_arr[0] + ',' + nicknames_arr[1] + ',' +nicknames_arr[2] + ',' + numOfDoc + ',' + subject_str + ',' + aff + ',' + city_name + ',' + count_name
                target_file.write(line.encode('utf-8'))
                target_file.write('\n')
        else:
            EmptyHtmlFiles_Log = open("ZERO_BYTE_HTML_LIST.txt", "a+")
            EmptyHtmlFiles_Log.write("author_" + str(htmlnumber) + ".html has zero byte!")
            EmptyHtmlFiles_Log.write("\n")
            EmptyHtmlFiles_Log.close()            
    except IOError:
        NoHtmlInputFile_Log = open("HTML_NOT_FOUND_LIST.txt", "a+")
        NoHtmlInputFile_Log.write("No input file: author_" + str(htmlnumber) + ".html")
        NoHtmlInputFile_Log.write("\n")
        NoHtmlInputFile_Log.close()
    authorId[:] = []
    authorNm[:] = []
    authorNickNames[:] = []
    documents[:] = []
    subjects[:] = []
    affiliations[:] = []
    city[:] = []
    country[:] = []
    authorUn[:] = []
    
    
for i in xrange(72091, 72092): #72091
    htmlToCsv(i)
    print i

72091
